In [ ]:
import sqlite3
import os
import pandas as pd

# Get relative path to the database file
db_path = "../amici/database/supreme_court_docs.db"

# Check if the file exists
if not os.path.exists(db_path):
    print(f"Database file not found at: {db_path}")
    print(f"Current working directory: {os.getcwd()}")
else:
    # Connect to SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    print(f"Successfully connected to SQLite database at {db_path}")

Successfully connected to SQLite database at ../amici/database/supreme_court_docs.db


## Testing the Connection

Once connected, you can test the connection by executing a simple query:

In [32]:
# Use this after connecting with either method above
try:
    # Sample query - adjust table name as needed
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    
    # Fetch and display results
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
except Exception as e:
    print(f"Error executing query: {e}")

Tables in the database:
- documents
- sqlite_sequence
- dockets
- amici
- lawyers


## Make excel sheets

In [ ]:
# Make excel sheets from the database
try:
    # Sample query to fetch data from a specific table
    cursor.execute("SELECT * FROM your_table_name;")  # Replace with your actual table name
    rows = cursor.fetchall()
    
    # Print the first few rows
    print("Sample data from the table:")
    for row in rows[:5]:  # Display first 5 rows
        print(row)
except Exception as e:
    print(f"Error fetching data: {e}")

In [33]:
try:
    # Query to find documents with complete_amici_list=False
    cursor.execute("SELECT * FROM documents WHERE complete_amici_list=0")
    
    # Fetch and display results
    documents_with_appendix = cursor.fetchall()
    
    # Get column names from cursor description
    column_names = [description[0] for description in cursor.description]
    
    # Print count of documents
    print(f"Found {len(documents_with_appendix)} documents with amici list incomplete.")
    
    # Display column names and first few results
    if documents_with_appendix:
        print("\nColumn names:", column_names)
        print("\nFirst 5 documents with incomplete amici:")
        for doc in documents_with_appendix[:5]:
            print(doc)
except Exception as e:
    print(f"Error executing query: {e}")

# Create a file storing the blobs of all documents with complete_amici_list=False
try:
    cursor.execute("SELECT * FROM documents WHERE complete_amici_list=0")
    documents_with_appendix = cursor.fetchall()
    
    # Open a file to write the blobs
    with open("../amici/data/incomplete_amici_blobs.txt", "w") as f:
        for doc in documents_with_appendix:
            # Assuming the blob is in the first column (index 0)
            f.write(doc[7]+'\n')
    
    print("Blobs of incomplete amici documents have been written to incomplete_amici_blobs.txt")
except Exception as e:
    print(f"Error writing blobs to file: {e}")
# Close the database connection
finally:
    if conn:
        conn.close()
        print("Database connection closed.")

Found 1179 documents with amici list incomplete.

Column names: ['document_id', 'url', 'docket_url', 'date', 'date_formatted', 'label', 'doc_title', 'blob', 'transcribed', 'neededOCR', 'complete_amici_list', 'counsel_of_record']

First 5 documents with incomplete amici:
(4, 'http://www.supremecourt.gov/DocketPDF/22/22-535/252021/20230111151208528_22-506and22-535tsacLawyersCommitteeForCivilRightsUnderLaw.pdf', 'www.supremecourt.gov/search.aspx?filename=/docket/docketfiles/html/public/22-535.html', 'Jan 11 2023', '2023-01-11', 'Brief amici curiae of Lawyers’ Committee For Civil Rights Under Law and 21 Other Organizations filed (also in 22-506).  VIDED.  (Distributed)', 'Main Document', 'SUPREMECOURT/www.supremecourt.gov/DocketPDF/22/22-535/252021/20230111151208528_22-506and22-535tsacLawyersCommitteeForCivilRightsUnderLaw.pdf', 1, 0, 0, 'Damon Hewitt')
(29, 'http://www.supremecourt.gov/DocketPDF/19/19-1392/185243/20210729123007530_41063%20pdf%20Pierce.pdf', 'www.supremecourt.gov/search.as